(algebra_2)=
# Quadratics Basics

<iframe 
    src="https://drive.google.com/file/d/1KQdEOxFP1FnUw8zJnDHHblAgbaGw_UCd/preview" 
    width="100%" 
    height="480" 
    frameborder="0" 
    allowfullscreen>
</iframe>

In [5]:
from IPython.display import HTML
import json

def load_math_explorer_lib():
    """
    Load the MathExplorer JavaScript library.
    This only needs to be called once per notebook.
    """
    # The content would be read from the external file in a real implementation
    with open('../_static/js/math_explorer.js', 'r') as f:
        js_content = f.read()
    
    html = f"""
    <script>
    {js_content}
    </script>
    """
    return HTML(html)

def create_explorer(container_id, title="Math Function Explorer", config=None):
    """
    Create an interactive function explorer.
    
    Parameters:
    - container_id: ID for the container div
    - title: Title for the explorer
    - config: Optional configuration dict with additional options
    
    Returns:
    - HTML object to display in the notebook
    """
    if config is None:
        config = {}
    
    options = {
        "title": title,
        "xMin": -10,
        "xMax": 10,
        "yMin": -10,
        "yMax": 10,
        "gridSpacing": 1,
        "padding": 30
    }
    options.update(config)
    
    # Format options as JavaScript object literals
    options_str = json.dumps(options)
    
    html_content = f"""
    <div id="{container_id}"></div>
    <script>
      // Create explorer when DOM is ready
      document.addEventListener('DOMContentLoaded', () => {{
        window.{container_id}_explorer = createFunctionExplorer('{container_id}', {options_str});
      }});
    </script>
    """
    
    return HTML(html_content)

def add_controls_script(container_id, controls, function_code, special_points=None, info_fields=None):
    """
    Add controls, function, and special points to the explorer.
    
    Parameters:
    - container_id: ID of the container div
    - controls: List of dicts with control parameters
    - function_code: JavaScript code that evaluates the function
    - special_points: List of dicts defining special points to show
    - info_fields: List of dicts defining information fields
    
    Returns:
    - HTML object with the script to add these elements
    """
    if special_points is None:
        special_points = []
    if info_fields is None:
        info_fields = []
        
    script = f"""
    <script>
      // Wait for DOM and explorer to be ready
      (function() {{
        function initialize() {{
          if (window.{container_id}_explorer) {{
            const explorer = window.{container_id}_explorer;
            
            // Add controls
            {_generate_controls_js(container_id, controls)}
            
            // Add info fields
            {_generate_info_fields_js(container_id, info_fields)}
            
            // Set function evaluator
            explorer.setFunction({function_code});
            
            // Add special points
            {_generate_special_points_js(container_id, special_points)}
            
            // Initial update
            explorer.updateGraph();
          }} else {{
            setTimeout(initialize, 100);
          }}
        }}
        
        // Start initialization when DOM is ready
        if (document.readyState === "complete" || document.readyState === "interactive") {{
          setTimeout(initialize, 0);
        }} else {{
          document.addEventListener("DOMContentLoaded", initialize);
        }}
      }})();
    </script>
    """
    
    return HTML(script)

def _generate_controls_js(container_id, controls):
    """Generate JavaScript to add controls to the explorer."""
    js_code = []
    for ctrl in controls:
        js_code.append(
            f"explorer.addControl('{ctrl['id']}', '{ctrl['label']}', {ctrl['min']}, "
            f"{ctrl['max']}, {ctrl['value']}, {ctrl['step']});"
        )
    return "\n            ".join(js_code)

def _generate_info_fields_js(container_id, info_fields):
    """Generate JavaScript to add info fields to the explorer."""
    js_code = []
    for field in info_fields:
        js_code.append(f"explorer.addInfoField('{field['id']}', '{field['label']}');")
    return "\n            ".join(js_code)

def _generate_special_points_js(container_id, special_points):
    """Generate JavaScript to add special points to the explorer."""
    js_code = []
    for point in special_points:
        js_code.append(
            f"explorer.addSpecialPoint({point['calculator']}, '{point['color']}');"
        )
    return "\n            ".join(js_code)

In [6]:
# Load the math explorer library (only needed once per notebook)
#from jupyter_math_explorer import load_math_explorer_lib, create_explorer, add_controls_script

# Load the library 
load_math_explorer_lib()

# Create a quadratic function explorer
create_explorer("quadratic-explorer", "Quadratic Function Explorer")

# Define controls, function, and special points
controls = [
    {"id": "a-param", "label": "a", "min": -3, "max": 3, "value": 1, "step": 0.1},
    {"id": "b-param", "label": "b", "min": -5, "max": 5, "value": 0, "step": 0.1},
    {"id": "c-param", "label": "c", "min": -5, "max": 5, "value": 0, "step": 0.1}
]

# Define info fields
info_fields = [
    {"id": "equation-display", "label": "Equation"},
    {"id": "roots-display", "label": "Roots"},
    {"id": "vertex-display", "label": "Vertex"},
    {"id": "y-intercept-display", "label": "Y-intercept"},
    {"id": "discriminant-display", "label": "Discriminant"}
]

# JavaScript function code (note the template string syntax)
function_code = """
(x, a, b, c) => {
    return a * x * x + b * x + c;
}
"""

# Special points calculators
special_points = [
    {
        "calculator": """
        (a, b, c) => {
            // Calculate vertex
            const vertexX = -b / (2 * a);
            const vertexY = a * vertexX * vertexX + b * vertexX + c;
            
            // Update info displays
            const discriminant = b * b - 4 * a * c;
            explorer.updateInfoField('discriminant-display', discriminant.toFixed(2));
            
            // Format equation
            let equation = 'y = ';
            if (a === 0) {
                if (b === 0) {
                    equation += c;
                } else {
                    equation += (b === 1) ? 'x' : (b === -1) ? '-x' : `${b}x`;
                    if (c !== 0) {
                        equation += c > 0 ? ` + ${c}` : ` - ${Math.abs(c)}`;
                    }
                }
            } else {
                equation += (a === 1) ? 'x²' : (a === -1) ? '-x²' : `${a}x²`;
                if (b !== 0) {
                    equation += (b === 1) ? ' + x' : (b === -1) ? ' - x' : 
                        (b > 0) ? ` + ${b}x` : ` - ${Math.abs(b)}x`;
                }
                if (c !== 0) {
                    equation += c > 0 ? ` + ${c}` : ` - ${Math.abs(c)}`;
                }
            }
            explorer.updateInfoField('equation-display', equation);
            
            // Format vertex
            explorer.updateInfoField('vertex-display', 
                `(${vertexX.toFixed(2)}, ${vertexY.toFixed(2)})`);
            
            // Format y-intercept
            explorer.updateInfoField('y-intercept-display', c.toFixed(2));
            
            // Calculate and format roots
            let rootsText;
            if (discriminant > 0) {
                const root1 = (-b + Math.sqrt(discriminant)) / (2 * a);
                const root2 = (-b - Math.sqrt(discriminant)) / (2 * a);
                rootsText = `x = ${root1.toFixed(2)} or x = ${root2.toFixed(2)}`;
            } else if (discriminant === 0) {
                const root = -b / (2 * a);
                rootsText = `x = ${root.toFixed(2)} (double root)`;
            } else {
                rootsText = "No real roots";
            }
            explorer.updateInfoField('roots-display', rootsText);
            
            return { x: vertexX, y: vertexY };
        }
        """,
        "color": "#e74c3c"
    },
    {
        "calculator": """
        (a, b, c) => {
            // Y-intercept point (0, c)
            return { x: 0, y: c };
        }
        """,
        "color": "#2ecc71"
    },
    {
        "calculator": """
        (a, b, c) => {
            // Roots (if they exist)
            const discriminant = b * b - 4 * a * c;
            
            if (discriminant < 0) return null;
            
            if (discriminant === 0) {
                const root = -b / (2 * a);
                return { x: root, y: 0 };
            } else {
                const root1 = (-b + Math.sqrt(discriminant)) / (2 * a);
                const root2 = (-b - Math.sqrt(discriminant)) / (2 * a);
                return [
                    { x: root1, y: 0 },
                    { x: root2, y: 0 }
                ];
            }
        }
        """,
        "color": "#9b59b6"
    }
]

# Add controls and complete setup
add_controls_script("quadratic-explorer", controls, function_code, special_points, info_fields)

FileNotFoundError: [Errno 2] No such file or directory: '../_static/js/math_explorer.js'

## Linear Equations Revision

Before diving into quadratic equations, it's worth taking a moment to review linear equations.

Linear equations are a class of polynomial[^1] equations that can be rearranged into the **standard form**:

$$ax + b = 0$$

where $a$ and $b$ are any constant numbers (with $a \neq 0$), and $x$ is the variable we're solving for[^2]. Once in this form, solving for $x$ requires just a couple of steps:

$$ax + b = 0$$
$$ax = -b$$
$$x = -\frac{b}{a}$$

We first isolate the $x$ term by subtracting $b$ from both sides, then divide by $a$ to get $x$ on its own.

Linear equations get their name because they create straight lines when graphed. They're also called "first-order equations" or "equations of degree 1" because the highest power of $x$ is 1 (since $x$ could also be written as $x^1$).

### Examples
#### Example 1
Let's solve: $3x + 9 = 3$

**Method 1: Converting to standard form first**

$$3x + 9 = 3 \quad \text{(Not in standard form yet)}$$

$$3x + 9 - 3 = 3 - 3 \quad \text{(Subtracting 3 from both sides)}$$

$$3x + 6 = 0 \quad \text{(Now in standard form)}$$

$$x = -\frac{6}{3} = -2 \quad \text{(Dividing by 3)}$$

**Method 2: Direct approach**

$$3x + 9 = 3 \quad \text{(Original equation)}$$

$$3x = 3 - 9 \quad \text{(Moving constant to the right)}$$

$$3x = -6 \quad \text{(Simplifying)}$$

$$x = -\frac{6}{3} = -2 \quad \text{(Dividing by 3)}$$

You might notice we took the long way round in Method 1 - we didn't really have to bring it to standard form first - and you'd be right! For a simple linear equation, we could just directly solve as in Method 2 if all we care about is finding the value of $x$.

However, for more advanced problems like quadratic equations, converting to standard form is almost always the first step to solving them. This habit will come in handy later, trust me.

[^1]: *Extra Learning*: Properly, polynomials are expressions of the form $a_n x^n + a_{n-1} x^{n-1} + \dots + a_1 x + a_0$, where the $a$'s are constants. In simple terms, if you take a bunch of x's, put them to various powers, multiply each by different numbers, and add them all together, you get a polynomial. Any definition more complicated than this is just mathematicians trying to make things more general (more general = more widely applicable = more useful), but we don't need to go deeper than this for now.

[^2]: Don't get hung up on the actual letter itself - you could technically use any letter you like.




In [ ]:
from IPython.display import HTML

# JavaScript-based interactive function explorer
# This will work completely client-side and offline once loaded

html_content = """
<div id="linear-function-explorer" style="font-family: 'Arial', sans-serif; max-width: 900px; margin: 0 auto; padding: 20px;">
    <h2 style="text-align: center; color: #2c3e50;">Linear Equation Explorer</h2>
    <div style="display: flex; flex-wrap: wrap; gap: 20px;">
        <div style="flex: 1; min-width: 300px;">
            <div style="margin-bottom: 20px;">
                <label for="linear-a-value">a: <span id="linear-a-value-display">1</span></label>
                <input type="range" id="linear-a-value" min="-5" max="5" value="1" step="0.1" style="width: 100%;">
            </div>
            <div style="margin-bottom: 20px;">
                <label for="linear-b-value">b: <span id="linear-b-value-display">0</span></label>
                <input type="range" id="linear-b-value" min="-5" max="5" value="0" step="0.1" style="width: 100%;">
            </div>
            <div style="margin-top: 30px; padding: 15px; background-color: #f8f9fa; border-radius: 5px; box-shadow: 0 1px 3px rgba(0,0,0,0.1);">
                <h3 style="margin-top: 0; color: #2c3e50;">Function Properties:</h3>
                <p>Standard Form: <span id="linear-standard-form-display">ax + b = 0</span></p>
                <p>Slope-Intercept Form: <span id="linear-slope-form-display">y = mx + c</span></p>
                <p>Slope: <span id="linear-slope-display">m</span></p>
                <p>X-intercept: <span id="linear-x-intercept-display">0</span></p>
                <p>Y-intercept: <span id="linear-y-intercept-display">0</span></p>
            </div>
        </div>
        <div style="flex: 2; min-width: 400px;">
            <div style="width: 100%; height: 400px; border: 1px solid #ddd; border-radius: 5px; box-shadow: 0 1px 3px rgba(0,0,0,0.1);">
                <canvas id="linear-graph-canvas" style="width: 100%; height: 100%;"></canvas>
            </div>
        </div>
    </div>
</div>

<script>
(function() {
    // Get canvas and context
    const linearCanvas = document.getElementById('linear-graph-canvas');
    const linearCtx = linearCanvas.getContext('2d');
    
    // Get DOM elements
    const linearASlider = document.getElementById('linear-a-value');
    const linearBSlider = document.getElementById('linear-b-value');
    const linearAValueDisplay = document.getElementById('linear-a-value-display');
    const linearBValueDisplay = document.getElementById('linear-b-value-display');
    const linearStandardFormDisplay = document.getElementById('linear-standard-form-display');
    const linearSlopeFormDisplay = document.getElementById('linear-slope-form-display');
    const linearSlopeDisplay = document.getElementById('linear-slope-display');
    const linearXInterceptDisplay = document.getElementById('linear-x-intercept-display');
    const linearYInterceptDisplay = document.getElementById('linear-y-intercept-display');
    
    // Set canvas dimensions to match its display size
    function linearResizeCanvas() {
        const displayWidth = linearCanvas.clientWidth;
        const displayHeight = linearCanvas.clientHeight;
        
        if (linearCanvas.width !== displayWidth || linearCanvas.height !== displayHeight) {
            linearCanvas.width = displayWidth;
            linearCanvas.height = displayHeight;
        }
    }
    linearResizeCanvas();
    window.addEventListener('resize', linearResizeCanvas);
    
    // Graph properties
    const linearGraphProps = {
        xMin: -10,
        xMax: 10,
        yMin: -10,
        yMax: 10,
        gridSpacing: 1,
        padding: 30
    };
    
    // Convert from math coordinates to canvas coordinates
    function linearToCanvasX(x) {
        return ((x - linearGraphProps.xMin) / (linearGraphProps.xMax - linearGraphProps.xMin)) * (linearCanvas.width - 2 * linearGraphProps.padding) + linearGraphProps.padding;
    }
    
    function linearToCanvasY(y) {
        return linearCanvas.height - (((y - linearGraphProps.yMin) / (linearGraphProps.yMax - linearGraphProps.yMin)) * (linearCanvas.height - 2 * linearGraphProps.padding) + linearGraphProps.padding);
    }
    
    // Linear function (in terms of y = mx + c)
    function linearFunction(x, a, b) {
        // Convert from standard form ax + b = 0 to slope-intercept form y = mx + c
        if (a === 0) {
            // Horizontal line (undefined slope)
            return b === 0 ? 0 : NaN; // If b is also 0, it's the x-axis, otherwise no solution
        }
        return (-a * x - b);
    }
    
    // Calculate linear function properties
    function linearCalculateProperties(a, b) {
        // For ax + b = 0
        
        // X-intercept (y = 0)
        const xIntercept = a === 0 ? (b === 0 ? "All x values" : "None") : -b / a;
        
        // Y-intercept (x = 0)
        const yIntercept = a === 0 ? (b === 0 ? 0 : NaN) : -b;
        
        // Slope (m) for y = mx + c form
        const slope = a === 0 ? "Undefined" : -a;
        
        return {
            xIntercept,
            yIntercept,
            slope
        };
    }
    
    // Draw the grid
    function linearDrawGrid() {
        linearCtx.strokeStyle = '#e1e1e1';
        linearCtx.lineWidth = 1;
        
        // Vertical grid lines
        for (let x = Math.ceil(linearGraphProps.xMin); x <= linearGraphProps.xMax; x += linearGraphProps.gridSpacing) {
            linearCtx.beginPath();
            linearCtx.moveTo(linearToCanvasX(x), 0);
            linearCtx.lineTo(linearToCanvasX(x), linearCanvas.height);
            linearCtx.stroke();
        }
        
        // Horizontal grid lines
        for (let y = Math.ceil(linearGraphProps.yMin); y <= linearGraphProps.yMax; y += linearGraphProps.gridSpacing) {
            linearCtx.beginPath();
            linearCtx.moveTo(0, linearToCanvasY(y));
            linearCtx.lineTo(linearCanvas.width, linearToCanvasY(y));
            linearCtx.stroke();
        }
        
        // Axes
        linearCtx.strokeStyle = '#000';
        linearCtx.lineWidth = 2;
        
        // X-axis
        linearCtx.beginPath();
        linearCtx.moveTo(linearToCanvasX(linearGraphProps.xMin), linearToCanvasY(0));
        linearCtx.lineTo(linearToCanvasX(linearGraphProps.xMax), linearToCanvasY(0));
        linearCtx.stroke();
        
        // Y-axis
        linearCtx.beginPath();
        linearCtx.moveTo(linearToCanvasX(0), linearToCanvasY(linearGraphProps.yMin));
        linearCtx.lineTo(linearToCanvasX(0), linearToCanvasY(linearGraphProps.yMax));
        linearCtx.stroke();
        
        // Axis labels
        linearCtx.fillStyle = '#000';
        linearCtx.font = '12px Arial';
        linearCtx.textAlign = 'center';
        
        // X-axis labels
        for (let x = Math.ceil(linearGraphProps.xMin); x <= linearGraphProps.xMax; x += linearGraphProps.gridSpacing) {
            if (x !== 0) {  // Skip zero to avoid overlapping with y-axis label
                linearCtx.fillText(x.toString(), linearToCanvasX(x), linearToCanvasY(0) + 15);
            }
        }
        
        // Y-axis labels
        linearCtx.textAlign = 'right';
        for (let y = Math.ceil(linearGraphProps.yMin); y <= linearGraphProps.yMax; y += linearGraphProps.gridSpacing) {
            if (y !== 0) {  // Skip zero to avoid overlapping with x-axis label
                linearCtx.fillText(y.toString(), linearToCanvasX(0) - 5, linearToCanvasY(y) + 4);
            }
        }
        
        // Origin label
        linearCtx.fillText("0", linearToCanvasX(0) - 5, linearToCanvasY(0) + 15);
    }
    
    // Draw the linear function
    function linearDrawFunction(a, b) {
        linearCtx.strokeStyle = '#3498db';
        linearCtx.lineWidth = 3;
        linearCtx.beginPath();
        
        if (a === 0) {
            // Handle horizontal line or no solution
            if (b === 0) {
                // Draw the x-axis (already drawn by grid, but in different color)
                linearCtx.moveTo(linearToCanvasX(linearGraphProps.xMin), linearToCanvasY(0));
                linearCtx.lineTo(linearToCanvasX(linearGraphProps.xMax), linearToCanvasY(0));
            }
            // If b ≠ 0, then equation is "b = 0" which has no solution, so we don't draw
        } else {
            // Regular line with defined slope
            const leftY = linearFunction(linearGraphProps.xMin, a, b);
            const rightY = linearFunction(linearGraphProps.xMax, a, b);
            
            linearCtx.moveTo(linearToCanvasX(linearGraphProps.xMin), linearToCanvasY(leftY));
            linearCtx.lineTo(linearToCanvasX(linearGraphProps.xMax), linearToCanvasY(rightY));
        }
        
        linearCtx.stroke();
    }
    
    // Draw special points
    function linearDrawSpecialPoints(a, b) {
        const props = linearCalculateProperties(a, b);
        
        // X-intercept
        if (typeof props.xIntercept === 'number') {
            linearCtx.fillStyle = '#e74c3c';
            linearCtx.beginPath();
            linearCtx.arc(linearToCanvasX(props.xIntercept), linearToCanvasY(0), 5, 0, 2 * Math.PI);
            linearCtx.fill();
        }
        
        // Y-intercept
        if (!isNaN(props.yIntercept)) {
            linearCtx.fillStyle = '#2ecc71';
            linearCtx.beginPath();
            linearCtx.arc(linearToCanvasX(0), linearToCanvasY(props.yIntercept), 5, 0, 2 * Math.PI);
            linearCtx.fill();
        }
    }
    
    // Format standard form equation string
    function linearFormatStandardForm(a, b) {
        if (a === 0 && b === 0) {
            return "0 = 0 (True for all x)";
        }
        if (a === 0) {
            return `${b} = 0`;
        }
        
        let equation = '';
        
        if (a === 1) {
            equation += 'x';
        } else if (a === -1) {
            equation += '-x';
        } else {
            equation += `${a}x`;
        }
        
        if (b !== 0) {
            equation += b > 0 ? ` + ${b}` : ` - ${Math.abs(b)}`;
        }
        
        equation += ' = 0';
        
        return equation;
    }
    
    // Format slope-intercept form equation string
    function linearFormatSlopeForm(a, b) {
        if (a === 0) {
            if (b === 0) {
                return "y = 0";
            }
            return "No slope-intercept form exists";
        }
        
        const slope = -a;
        const yIntercept = -b / a;
        
        let equation = 'y = ';
        
        if (slope === 1) {
            equation += 'x';
        } else if (slope === -1) {
            equation += '-x';
        } else {
            equation += `${slope}x`;
        }
        
        if (yIntercept !== 0) {
            equation += yIntercept > 0 ? ` + ${yIntercept}` : ` - ${Math.abs(yIntercept)}`;
        }
        
        return equation;
    }
    
    // Update the graph and information
    function linearUpdateGraph() {
        // Get current slider values
        const a = parseFloat(linearASlider.value);
        const b = parseFloat(linearBSlider.value);
        
        // Clear canvas
        linearCtx.clearRect(0, 0, linearCanvas.width, linearCanvas.height);
        
        // Draw grid and function
        linearDrawGrid();
        linearDrawFunction(a, b);
        linearDrawSpecialPoints(a, b);
        
        // Calculate properties
        const props = linearCalculateProperties(a, b);
        
        // Format x-intercept for display
        let xInterceptText;
        if (typeof props.xIntercept === 'number') {
            xInterceptText = props.xIntercept.toFixed(2);
        } else {
            xInterceptText = props.xIntercept;
        }
        
        // Update displays
        linearAValueDisplay.textContent = a.toFixed(1);
        linearBValueDisplay.textContent = b.toFixed(1);
        linearStandardFormDisplay.textContent = linearFormatStandardForm(a, b);
        linearSlopeFormDisplay.textContent = linearFormatSlopeForm(a, b);
        linearSlopeDisplay.textContent = typeof props.slope === 'number' ? props.slope.toFixed(2) : props.slope;
        linearXInterceptDisplay.textContent = xInterceptText;
        linearYInterceptDisplay.textContent = isNaN(props.yIntercept) ? "None" : props.yIntercept.toFixed(2);
    }
    
    // Event listeners for sliders
    linearASlider.addEventListener('input', linearUpdateGraph);
    linearBSlider.addEventListener('input', linearUpdateGraph);
    
    // Initial render
    linearUpdateGraph();
})();
</script>"""

# Display the HTML content
HTML(html_content)

## Quadratic Equations and their Form

Quadratic equations are the next step up from linear equations in the polynomial family. They have the following **standard form**:

$$ax^2 + bx + c = 0$$

where $a$, $b$, and $c$ are constants (with $a \neq 0$), and $x$ is our variable. What makes an equation quadratic is the presence of that $x^2$ term — the **highest** power of $x$ must be 2.

Getting quadratic equations into this standard form serves two main purposes:
1. It's the starting point for solving them using standard techniques
2. The coefficients $a$, $b$, and $c$ reveal important information about the graph of the equation

If you can transform another type of equation into quadratic form without violating any mathematical rules, you can use quadratic techniques to solve problems that might otherwise seem challenging.

### Examples: Identifying and Working with Quadratic Equations

#### Example 1: Expanding brackets

Is this a quadratic equation?

$$2x(x+3) = -x$$

To find out, we'll expand the brackets and bring everything to one side:

$$2x(x+3) = -x$$
$$2x^2 + 6x = -x$$
$$2x^2 + 7x = 0$$

Yes! This is a quadratic equation where $a=2$, $b=7$, and $c=0$.

#### Example 2: Substitution method

Is this a quadratic equation?

$$x^4 + x^2 + 1 = 0$$

At first glance, no—the highest power is 4, not 2. However, we can use a clever substitution. Let's define $t = x^2$ and see what happens:

$$x^4 + x^2 + 1 = 0$$
$$(x^2)^2 + x^2 + 1 = 0$$
$$t^2 + t + 1 = 0$$

We've transformed it into a quadratic equation in terms of $t$, where $a=1$, $b=1$, and $c=1$. After solving for $t$, we can find $x$ using $t = x^2$. This substitution trick appears frequently in exams and is worth mastering.

#### Example 3: Trigonometric substitution

$$\tan^2(x) + \tan(x) + 2 = 0$$

This follows the same logic as Example 2. We can let $t = \tan(x)$ and substitute:

$$\tan^2(x) + \tan(x) + 2 = 0$$
$$t^2 + t + 2 = 0$$

Now we have a quadratic in $t$ with $a=1$, $b=1$, and $c=2$.

#### Example 4: Not all equations with $x^2$ are quadratic

$$\tan(x) + x^2 + 5 = 0$$

Despite having an $x^2$ term, this is not a quadratic equation. If we try to substitute $t = \tan(x)$:

$$t + x^2 + 5 = 0$$

We end up with two different variables ($t$ and $x$), and no way to convert between them. The presence of the trigonometric function $\tan(x)$ alongside the polynomial term $x^2$ prevents this from being converted to quadratic form.

#### Example 5: Functions with constants

$$x^2\sin\left(\frac{\pi}{3}\right) + x = 0$$

Although it involves a trigonometric term, this **is** a quadratic equation because $\sin\left(\frac{\pi}{3}\right)$ evaluates to a constant (approximately 0.866). The equation has the form:

$$ax^2 + bx + c = 0$$

where $a = \sin\left(\frac{\pi}{3}\right)$, $b = 1$, and $c = 0$.

### Key Takeaways

1. A quadratic equation has the form $ax^2 + bx + c = 0$ where $a \neq 0$
2. Equations can often be rewritten or transformed into quadratic form
3. Using substitution, we can convert some higher-degree equations into quadratics
4. Not all equations containing $x^2$ are quadratics, especially if they contain other non-polynomial functions of $x$
5. A trigonometric expression that doesn't contain $x$ as a variable acts as a constant in the equation

In [ ]:
from IPython.display import HTML

# JavaScript-based interactive function explorer
# This will work completely client-side and offline once loaded

html_content = """
<div id="function-explorer" style="font-family: 'Arial', sans-serif; max-width: 900px; margin: 0 auto; padding: 20px;">
    <h2 style="text-align: center; color: #2c3e50;">Quadratic Function Explorer</h2>
    <div style="display: flex; flex-wrap: wrap; gap: 20px;">
        <div style="flex: 1; min-width: 300px;">
            <div style="margin-bottom: 20px;">
                <label for="a-value">a: <span id="a-value-display">1</span></label>
                <input type="range" id="a-value" min="-3" max="3" value="1" step="0.1" style="width: 100%;">
            </div>
            <div style="margin-bottom: 20px;">
                <label for="b-value">b: <span id="b-value-display">0</span></label>
                <input type="range" id="b-value" min="-5" max="5" value="0" step="0.1" style="width: 100%;">
            </div>
            <div style="margin-bottom: 20px;">
                <label for="c-value">c: <span id="c-value-display">0</span></label>
                <input type="range" id="c-value" min="-5" max="5" value="0" step="0.1" style="width: 100%;">
            </div>
            <div style="margin-top: 30px; padding: 15px; background-color: #f8f9fa; border-radius: 5px; box-shadow: 0 1px 3px rgba(0,0,0,0.1);">
                <h3 style="margin-top: 0; color: #2c3e50;">Function Properties:</h3>
                <p>Equation: <span id="equation-display">y = x²</span></p>
                <p>Roots: <span id="roots-display">x = 0</span></p>
                <p>Vertex: <span id="vertex-display">(0, 0)</span></p>
                <p>Y-intercept: <span id="y-intercept-display">0</span></p>
                <p>Discriminant: <span id="discriminant-display">0</span></p>
            </div>
        </div>
        <div style="flex: 2; min-width: 400px;">
            <div style="width: 100%; height: 400px; border: 1px solid #ddd; border-radius: 5px; box-shadow: 0 1px 3px rgba(0,0,0,0.1);">
                <canvas id="graph-canvas" style="width: 100%; height: 100%;"></canvas>
            </div>
        </div>
    </div>
</div>

<script>
(function() {
    // Get canvas and context
    const canvas = document.getElementById('graph-canvas');
    const ctx = canvas.getContext('2d');
    
    // Get DOM elements
    const aSlider = document.getElementById('a-value');
    const bSlider = document.getElementById('b-value');
    const cSlider = document.getElementById('c-value');
    const aValueDisplay = document.getElementById('a-value-display');
    const bValueDisplay = document.getElementById('b-value-display');
    const cValueDisplay = document.getElementById('c-value-display');
    const equationDisplay = document.getElementById('equation-display');
    const rootsDisplay = document.getElementById('roots-display');
    const vertexDisplay = document.getElementById('vertex-display');
    const yInterceptDisplay = document.getElementById('y-intercept-display');
    const discriminantDisplay = document.getElementById('discriminant-display');
    
    // Set canvas dimensions to match its display size
    function resizeCanvas() {
        const displayWidth = canvas.clientWidth;
        const displayHeight = canvas.clientHeight;
        
        if (canvas.width !== displayWidth || canvas.height !== displayHeight) {
            canvas.width = displayWidth;
            canvas.height = displayHeight;
        }
    }
    resizeCanvas();
    window.addEventListener('resize', resizeCanvas);
    
    // Graph properties
    const graphProps = {
        xMin: -10,
        xMax: 10,
        yMin: -10,
        yMax: 10,
        gridSpacing: 1,
        padding: 30
    };
    
    // Convert from math coordinates to canvas coordinates
    function toCanvasX(x) {
        return ((x - graphProps.xMin) / (graphProps.xMax - graphProps.xMin)) * (canvas.width - 2 * graphProps.padding) + graphProps.padding;
    }
    
    function toCanvasY(y) {
        return canvas.height - (((y - graphProps.yMin) / (graphProps.yMax - graphProps.yMin)) * (canvas.height - 2 * graphProps.padding) + graphProps.padding);
    }
    
    // Quadratic function
    function quadratic(x, a, b, c) {
        return a * x * x + b * x + c;
    }
    
    // Calculate quadratic function properties
    function calculateProperties(a, b, c) {
        // Discriminant
        const discriminant = b * b - 4 * a * c;
        
        // Roots
        let rootsText = "";
        if (discriminant > 0) {
            const root1 = (-b + Math.sqrt(discriminant)) / (2 * a);
            const root2 = (-b - Math.sqrt(discriminant)) / (2 * a);
            rootsText = `x = ${root1.toFixed(2)} or x = ${root2.toFixed(2)}`;
        } else if (discriminant === 0) {
            const root = -b / (2 * a);
            rootsText = `x = ${root.toFixed(2)} (double root)`;
        } else {
            rootsText = "No real roots";
        }
        
        // Vertex
        const vertexX = -b / (2 * a);
        const vertexY = quadratic(vertexX, a, b, c);
        
        // Y-intercept
        const yIntercept = c;
        
        return {
            discriminant,
            rootsText,
            vertex: { x: vertexX, y: vertexY },
            yIntercept
        };
    }
    
    // Draw the grid
    function drawGrid() {
        ctx.strokeStyle = '#e1e1e1';
        ctx.lineWidth = 1;
        
        // Vertical grid lines
        for (let x = Math.ceil(graphProps.xMin); x <= graphProps.xMax; x += graphProps.gridSpacing) {
            ctx.beginPath();
            ctx.moveTo(toCanvasX(x), 0);
            ctx.lineTo(toCanvasX(x), canvas.height);
            ctx.stroke();
        }
        
        // Horizontal grid lines
        for (let y = Math.ceil(graphProps.yMin); y <= graphProps.yMax; y += graphProps.gridSpacing) {
            ctx.beginPath();
            ctx.moveTo(0, toCanvasY(y));
            ctx.lineTo(canvas.width, toCanvasY(y));
            ctx.stroke();
        }
        
        // Axes
        ctx.strokeStyle = '#000';
        ctx.lineWidth = 2;
        
        // X-axis
        ctx.beginPath();
        ctx.moveTo(toCanvasX(graphProps.xMin), toCanvasY(0));
        ctx.lineTo(toCanvasX(graphProps.xMax), toCanvasY(0));
        ctx.stroke();
        
        // Y-axis
        ctx.beginPath();
        ctx.moveTo(toCanvasX(0), toCanvasY(graphProps.yMin));
        ctx.lineTo(toCanvasX(0), toCanvasY(graphProps.yMax));
        ctx.stroke();
        
        // Axis labels
        ctx.fillStyle = '#000';
        ctx.font = '12px Arial';
        ctx.textAlign = 'center';
        
        // X-axis labels
        for (let x = Math.ceil(graphProps.xMin); x <= graphProps.xMax; x += graphProps.gridSpacing) {
            if (x !== 0) {  // Skip zero to avoid overlapping with y-axis label
                ctx.fillText(x.toString(), toCanvasX(x), toCanvasY(0) + 15);
            }
        }
        
        // Y-axis labels
        ctx.textAlign = 'right';
        for (let y = Math.ceil(graphProps.yMin); y <= graphProps.yMax; y += graphProps.gridSpacing) {
            if (y !== 0) {  // Skip zero to avoid overlapping with x-axis label
                ctx.fillText(y.toString(), toCanvasX(0) - 5, toCanvasY(y) + 4);
            }
        }
        
        // Origin label
        ctx.fillText("0", toCanvasX(0) - 5, toCanvasY(0) + 15);
    }
    
    // Draw the quadratic function
    function drawFunction(a, b, c) {
        ctx.strokeStyle = '#3498db';
        ctx.lineWidth = 3;
        ctx.beginPath();
        
        const step = (graphProps.xMax - graphProps.xMin) / canvas.width;
        
        for (let x = graphProps.xMin; x <= graphProps.xMax; x += step) {
            const y = quadratic(x, a, b, c);
            
            if (y >= graphProps.yMin && y <= graphProps.yMax) {
                if (x === graphProps.xMin) {
                    ctx.moveTo(toCanvasX(x), toCanvasY(y));
                } else {
                    ctx.lineTo(toCanvasX(x), toCanvasY(y));
                }
            }
        }
        
        ctx.stroke();
    }
    
    // Draw special points
    function drawSpecialPoints(a, b, c) {
        const props = calculateProperties(a, b, c);
        
        // Vertex
        ctx.fillStyle = '#e74c3c';
        ctx.beginPath();
        ctx.arc(toCanvasX(props.vertex.x), toCanvasY(props.vertex.y), 5, 0, 2 * Math.PI);
        ctx.fill();
        
        // Y-intercept
        ctx.fillStyle = '#2ecc71';
        ctx.beginPath();
        ctx.arc(toCanvasX(0), toCanvasY(props.yIntercept), 5, 0, 2 * Math.PI);
        ctx.fill();
        
        // Roots (if they exist)
        if (props.discriminant >= 0) {
            ctx.fillStyle = '#9b59b6';
            
            if (props.discriminant > 0) {
                const root1 = (-b + Math.sqrt(props.discriminant)) / (2 * a);
                const root2 = (-b - Math.sqrt(props.discriminant)) / (2 * a);
                
                ctx.beginPath();
                ctx.arc(toCanvasX(root1), toCanvasY(0), 5, 0, 2 * Math.PI);
                ctx.fill();
                
                ctx.beginPath();
                ctx.arc(toCanvasX(root2), toCanvasY(0), 5, 0, 2 * Math.PI);
                ctx.fill();
            } else {
                const root = -b / (2 * a);
                
                ctx.beginPath();
                ctx.arc(toCanvasX(root), toCanvasY(0), 5, 0, 2 * Math.PI);
                ctx.fill();
            }
        }
    }
    
    // Format equation string
    function formatEquation(a, b, c) {
        let equation = 'y = ';
        
        if (a === 0) {
            if (b === 0) {
                equation += c;
            } else {
                if (b === 1) {
                    equation += 'x';
                } else if (b === -1) {
                    equation += '-x';
                } else {
                    equation += `${b}x`;
                }
                
                if (c !== 0) {
                    equation += c > 0 ? ` + ${c}` : ` - ${Math.abs(c)}`;
                }
            }
        } else {
            if (a === 1) {
                equation += 'x²';
            } else if (a === -1) {
                equation += '-x²';
            } else {
                equation += `${a}x²`;
            }
            
            if (b !== 0) {
                if (b === 1) {
                    equation += ' + x';
                } else if (b === -1) {
                    equation += ' - x';
                } else {
                    equation += b > 0 ? ` + ${b}x` : ` - ${Math.abs(b)}x`;
                }
            }
            
            if (c !== 0) {
                equation += c > 0 ? ` + ${c}` : ` - ${Math.abs(c)}`;
            }
        }
        
        return equation;
    }
    
    // Update the graph and information
    function updateGraph() {
        // Get current slider values
        const a = parseFloat(aSlider.value);
        const b = parseFloat(bSlider.value);
        const c = parseFloat(cSlider.value);
        
        // Clear canvas
        ctx.clearRect(0, 0, canvas.width, canvas.height);
        
        // Draw grid and function
        drawGrid();
        drawFunction(a, b, c);
        drawSpecialPoints(a, b, c);
        
        // Calculate properties
        const props = calculateProperties(a, b, c);
        
        // Update displays
        aValueDisplay.textContent = a.toFixed(1);
        bValueDisplay.textContent = b.toFixed(1);
        cValueDisplay.textContent = c.toFixed(1);
        equationDisplay.textContent = formatEquation(a, b, c);
        rootsDisplay.textContent = props.rootsText;
        vertexDisplay.textContent = `(${props.vertex.x.toFixed(2)}, ${props.vertex.y.toFixed(2)})`;
        yInterceptDisplay.textContent = props.yIntercept.toFixed(2);
        discriminantDisplay.textContent = props.discriminant.toFixed(2);
    }
    
    // Event listeners for sliders
    aSlider.addEventListener('input', updateGraph);
    bSlider.addEventListener('input', updateGraph);
    cSlider.addEventListener('input', updateGraph);
    
    // Initial render
    updateGraph();
})();
</script>
"""

# Display the HTML content
HTML(html_content)

## Factorizing Quadratic Equations by Inspection

Factorizing by inspection is the most intuitive approach to solving quadratic equations. At its core, factorization means breaking down an expression into simpler parts that multiply together to give the original expression.

### Understanding the Concept

When we have a quadratic equation in standard form:

$$ax^2 + bx + c = 0$$

Factorizing means rewriting it as:

$$(px + q)(rx + s) = 0$$

Where $p$, $q$, $r$, and $s$ are constants we need to find. The beauty of this approach is that once factorized, the solutions become immediately apparent thanks to the **zero product property**: if a product equals zero, at least one of the factors must be zero.

Factorizing by inspection works best when:
- The coefficients are "nice" integers (no messy fractions or surds)
- You have strong pattern recognition skills
- You need a quick solution without using formulas

If you see fractions, decimals, or square roots in your coefficients, you're probably better off using the quadratic formula directly.

### The Factorizing Process

Let's break down the approach:

1. Ensure your equation is in standard form with zero on the right side
2. Look for two numbers that:
   - Multiply to give $ac$ (the product of the first and last coefficients)
   - Add up to give $b$ (the middle coefficient)
3. Use these numbers to split the middle term and create your factors

### Example: Step by Step

Let's work through:

$$x^2 + 2x - 8 = 0$$

**Step 1:** We need to find two numbers that multiply to give $-8$ (which is $a \times c = 1 \times -8$) and add up to give $+2$ (our $b$ value).

**Step 2:** Let's list possible number pairs that multiply to give $-8$:
- $1 \times -8 = -8$ (sum: $1 + (-8) = -7$)
- $2 \times -4 = -8$ (sum: $2 + (-4) = -2$)
- $-1 \times 8 = -8$ (sum: $-1 + 8 = 7$)
- $-2 \times 4 = -8$ (sum: $-2 + 4 = 2$) ← This works!

**Step 3:** Now we've found our pair: $-2$ and $4$. Let's use them to split the middle term:
$x^2 + 2x - 8 = x^2 - 2x + 4x - 8$

**Step 4:** Group the terms in pairs and factor each pair:
$x^2 - 2x + 4x - 8 = x(x - 2) + 4(x - 2)$

**Step 5:** Factor out the common term $(x - 2)$:
$x(x - 2) + 4(x - 2) = (x + 4)(x - 2)$

So our factorized equation is:
$$(x + 4)(x - 2) = 0$$

#### Finding the Solutions

Now that we have factorized the equation, the solutions are straightforward:

If $(x + 4)(x - 2) = 0$, then either:
- $x + 4 = 0$, which gives $x = -4$
- $x - 2 = 0$, which gives $x = 2$

We can verify these solutions:
- For $x = -4$: $(-4)^2 + 2(-4) - 8 = 16 - 8 - 8 = 0$ ✓
- For $x = 2$: $(2)^2 + 2(2) - 8 = 4 + 4 - 8 = 0$ ✓

$x = 2$ and $x = -4$ are the **roots** of the quadratic equation.

#### When the Leading Coefficient Isn't 1

When the coefficient of $x^2$ isn't 1 (i.e., $a \neq 1$), factorizing by inspection becomes more challenging because we have more combinations to consider.

Two approaches for these cases:
1. First multiply everything by $a$ to make the coefficient of $x^2$ equal to $a^2$, then find factors where the outer terms multiply to give $a^2 \times c$
2. Use the quadratic formula directly (which we'll cover in a later section)

### Key Takeaway

Factorizing by inspection relies on pattern recognition and understanding the relationship between the coefficients and the factors. While it's not always applicable, it's often the quickest method when it works. As you practice, you'll develop an intuition for when to use this approach versus the quadratic formula.

In [ ]:
html_content="""
<div id="factorization-explorer" style="font-family: 'Arial', sans-serif; max-width: 900px; margin: 0 auto; padding: 20px;">
    <h2 style="text-align: center; color: #2c3e50;">Quadratic Factorization Explorer</h2>
    <div style="display: flex; flex-wrap: wrap; gap: 20px;">
        <div style="flex: 1; min-width: 300px;">
            <div style="margin-bottom: 20px;">
                <h3>Enter Quadratic Equation:</h3>
                <div style="display: flex; align-items: center; gap: 5px; margin-bottom: 15px;">
                    <input type="number" id="factor-a-value" value="1" style="width: 60px; padding: 5px; border: 1px solid #ccc; border-radius: 4px;">
                    <label>x² + </label>
                    <input type="number" id="factor-b-value" value="2" style="width: 60px; padding: 5px; border: 1px solid #ccc; border-radius: 4px;">
                    <label>x + </label>
                    <input type="number" id="factor-c-value" value="-8" style="width: 60px; padding: 5px; border: 1px solid #ccc; border-radius: 4px;">
                    <label>= 0</label>
                </div>
                <button id="factor-update-btn" style="padding: 8px 15px; background-color: #3498db; color: white; border: none; border-radius: 4px; cursor: pointer;">Update Equation</button>
            </div>
            
            <div style="margin-top: 20px; background-color: #f8f9fa; border-radius: 5px; padding: 15px; box-shadow: 0 1px 3px rgba(0,0,0,0.1);">
                <h3 style="margin-top: 0; color: #2c3e50;">Try to Factorize:</h3>
                <p id="factor-equation-display" style="font-weight: bold; font-size: 18px;">x² + 2x - 8 = 0</p>
                <p style="margin-bottom: 15px;">Enter your factorization:</p>
                <div style="display: flex; align-items: center; gap: 5px; margin-bottom: 15px;">
                    <label>(</label>
                    <input type="number" id="factor-p1" value="1" style="width: 40px; padding: 5px; border: 1px solid #ccc; border-radius: 4px;">
                    <label>x + </label>
                    <input type="number" id="factor-q1" value="4" style="width: 40px; padding: 5px; border: 1px solid #ccc; border-radius: 4px;">
                    <label>)(</label>
                    <input type="number" id="factor-p2" value="1" style="width: 40px; padding: 5px; border: 1px solid #ccc; border-radius: 4px;">
                    <label>x </label>
                    <select id="factor-sign" style="padding: 5px; border: 1px solid #ccc; border-radius: 4px;">
                        <option value="+">+</option>
                        <option value="-" selected>-</option>
                    </select>
                    <input type="number" id="factor-q2" value="2" style="width: 40px; padding: 5px; border: 1px solid #ccc; border-radius: 4px;">
                    <label>) = 0</label>
                </div>
                <button id="factor-check-btn" style="padding: 8px 15px; background-color: #2ecc71; color: white; border: none; border-radius: 4px; cursor: pointer;">Check Factorization</button>
            </div>
            
            <div id="factor-feedback" style="margin-top: 20px; padding: 15px; background-color: #e8f4fc; border-radius: 5px; box-shadow: 0 1px 3px rgba(0,0,0,0.1);">
                <h3 style="margin-top: 0; color: #2c3e50;">Feedback:</h3>
                <p id="factor-feedback-text">Enter your factorization above and click "Check Factorization"</p>
            </div>
        </div>
        
        <div style="flex: 1; min-width: 400px;">
            <div style="background-color: #f8f9fa; border-radius: 5px; padding: 15px; box-shadow: 0 1px 3px rgba(0,0,0,0.1); margin-bottom: 20px;">
                <h3 style="margin-top: 0; color: #2c3e50;">Finding Possible Factors</h3>
                <p>For the equation <span id="factor-hint-equation">x² + 2x - 8 = 0</span>, we need to find two numbers that:</p>
                <ol>
                    <li>Multiply to give <span id="factor-ac">-8</span> (product of a × c)</li>
                    <li>Add up to give <span id="factor-b">2</span> (middle coefficient b)</li>
                </ol>
                
                <table style="width: 100%; border-collapse: collapse; margin-top: 15px;">
                    <thead>
                        <tr style="background-color: #e0e0e0;">
                            <th style="padding: 8px; text-align: left; border: 1px solid #ccc;">Factor Pairs of <span id="factor-table-ac">-8</span></th>
                            <th style="padding: 8px; text-align: center; border: 1px solid #ccc;">Sum</th>
                            <th style="padding: 8px; text-align: center; border: 1px solid #ccc;">Works?</th>
                        </tr>
                    </thead>
                    <tbody id="factor-pairs-table">
                        <!-- Will be populated by JavaScript -->
                    </tbody>
                </table>
            </div>
            
            <div id="solution-steps" style="background-color: #f8f9fa; border-radius: 5px; padding: 15px; box-shadow: 0 1px 3px rgba(0,0,0,0.1); display: none;">
                <h3 style="margin-top: 0; color: #2c3e50;">Solution Steps:</h3>
                <ol id="factor-steps-list">
                    <!-- Will be populated by JavaScript -->
                </ol>
                
                <h3 style="color: #2c3e50;">Roots:</h3>
                <p id="factor-roots"></p>
                
                <h3 style="color: #2c3e50;">Verification:</h3>
                <div id="factor-verification"></div>
            </div>
        </div>
    </div>
</div>

<script>
(function() {
    // Get DOM elements
    const aInput = document.getElementById('factor-a-value');
    const bInput = document.getElementById('factor-b-value');
    const cInput = document.getElementById('factor-c-value');
    const updateBtn = document.getElementById('factor-update-btn');
    const equationDisplay = document.getElementById('factor-equation-display');
    const hintEquation = document.getElementById('factor-hint-equation');
    
    const p1Input = document.getElementById('factor-p1');
    const q1Input = document.getElementById('factor-q1');
    const p2Input = document.getElementById('factor-p2');
    const q2Input = document.getElementById('factor-q2');
    const signSelect = document.getElementById('factor-sign');
    const checkBtn = document.getElementById('factor-check-btn');
    
    const feedbackText = document.getElementById('factor-feedback-text');
    const solutionSteps = document.getElementById('solution-steps');
    const stepsList = document.getElementById('factor-steps-list');
    const rootsDisplay = document.getElementById('factor-roots');
    const verificationDisplay = document.getElementById('factor-verification');
    
    const acProduct = document.getElementById('factor-ac');
    const bValue = document.getElementById('factor-b');
    const tableAcProduct = document.getElementById('factor-table-ac');
    const factorPairsTable = document.getElementById('factor-pairs-table');
    
    // Format a quadratic equation nicely
    function formatEquation(a, b, c) {
        let equation = '';
        
        if (a === 0) {
            return 'Not a quadratic equation (a = 0)';
        }
        
        if (a === 1) {
            equation = 'x²';
        } else if (a === -1) {
            equation = '-x²';
        } else {
            equation = `${a}x²`;
        }
        
        if (b !== 0) {
            if (b > 0) {
                equation += ` + ${b === 1 ? '' : b}x`;
            } else {
                equation += ` - ${Math.abs(b) === 1 ? '' : Math.abs(b)}x`;
            }
        }
        
        if (c !== 0) {
            if (c > 0) {
                equation += ` + ${c}`;
            } else {
                equation += ` - ${Math.abs(c)}`;
            }
        }
        
        equation += ' = 0';
        return equation;
    }
    
    // Find all factor pairs for a number
    function findFactorPairs(num) {
        const pairs = [];
        const absNum = Math.abs(num);
        
        for (let i = 1; i <= Math.sqrt(absNum); i++) {
            if (absNum % i === 0) {
                const j = absNum / i;
                
                // If number is negative, we need one positive and one negative factor
                if (num < 0) {
                    pairs.push([i, -j]);
                    pairs.push([-i, j]);
                } else {
                    pairs.push([i, j]);
                    pairs.push([-i, -j]);
                }
            }
        }
        
        return pairs;
    }
    
    // Update the equation and factor hints
    function updateEquation() {
        const a = parseInt(aInput.value) || 1;
        const b = parseInt(bInput.value) || 0;
        const c = parseInt(cInput.value) || 0;
        
        // Format and display the equation
        const equation = formatEquation(a, b, c);
        equationDisplay.textContent = equation;
        hintEquation.textContent = equation;
        
        // Update the factor hints
        const ac = a * c;
        acProduct.textContent = ac;
        bValue.textContent = b;
        tableAcProduct.textContent = ac;
        
        // Generate and display factor pairs
        updateFactorPairs(a, b, c);
        
        // Reset solution display
        solutionSteps.style.display = 'none';
        feedbackText.textContent = 'Enter your factorization above and click "Check Factorization"';
        feedbackText.style.color = 'black';
    }
    
    // Update the factor pairs table
    function updateFactorPairs(a, b, c) {
        factorPairsTable.innerHTML = '';
        
        if (a === 0) {
            const row = document.createElement('tr');
            row.innerHTML = `
                <td style="padding: 8px; border: 1px solid #ccc;">Not applicable (a = 0)</td>
                <td style="padding: 8px; border: 1px solid #ccc; text-align: center;">-</td>
                <td style="padding: 8px; border: 1px solid #ccc; text-align: center;">-</td>
            `;
            factorPairsTable.appendChild(row);
            return;
        }
        
        const ac = a * c;
        
        if (ac === 0) {
            const row = document.createElement('tr');
            row.innerHTML = `
                <td style="padding: 8px; border: 1px solid #ccc;">0 × any number = 0</td>
                <td style="padding: 8px; border: 1px solid #ccc; text-align: center;">-</td>
                <td style="padding: 8px; border: 1px solid #ccc; text-align: center;">-</td>
            `;
            factorPairsTable.appendChild(row);
            return;
        }
        
        // Get all factor pairs
        const factorPairs = findFactorPairs(ac);
        
        // For each pair, check if their sum equals b
        factorPairs.forEach(pair => {
            const sum = pair[0] + pair[1];
            const works = sum === b;
            
            const row = document.createElement('tr');
            row.innerHTML = `
                <td style="padding: 8px; border: 1px solid #ccc;">${pair[0]} × ${pair[1]} = ${ac}</td>
                <td style="padding: 8px; border: 1px solid #ccc; text-align: center;">${sum}</td>
                <td style="padding: 8px; border: 1px solid #ccc; text-align: center; font-weight: bold; color: ${works ? 'green' : 'red'};">
                    ${works ? '✓' : '✗'}
                </td>
            `;
            
            if (works) {
                row.style.backgroundColor = '#e6ffe6';
            }
            
            factorPairsTable.appendChild(row);
        });
    }
    
    // Check the user's factorization
    function checkFactorization() {
        const a = parseInt(aInput.value) || 1;
        const b = parseInt(bInput.value) || 0;
        const c = parseInt(cInput.value) || 0;
        
        const p1 = parseInt(p1Input.value) || 1;
        const q1 = parseInt(q1Input.value) || 0;
        const p2 = parseInt(p2Input.value) || 1;
        let q2 = parseInt(q2Input.value) || 0;
        
        // Apply sign to q2
        if (signSelect.value === '-') {
            q2 = -Math.abs(q2);
        } else {
            q2 = Math.abs(q2);
        }
        
        // Calculate expected coefficients from factorization
        const expandedA = p1 * p2;
        const expandedB = p1 * q2 + p2 * q1;
        const expandedC = q1 * q2;
        
        // Check if factorization is correct
        const isCorrect = expandedA === a && expandedB === b && expandedC === c;
        
        if (isCorrect) {
            feedbackText.textContent = 'Correct! Your factorization is valid.';
            feedbackText.style.color = 'green';
            showSolution(a, b, c, p1, q1, p2, q2);
        } else {
            feedbackText.textContent = `Incorrect. Your factorization expands to ${formatEquation(expandedA, expandedB, expandedC)}`;
            feedbackText.style.color = 'red';
            solutionSteps.style.display = 'none';
        }
    }
    
    // Format a factor expression
    function formatFactor(p, q) {
        if (p === 0) return q.toString();
        
        let factor = '';
        
        if (p === 1) {
            factor = 'x';
        } else if (p === -1) {
            factor = '-x';
        } else {
            factor = `${p}x`;
        }
        
        if (q > 0) {
            factor += ` + ${q}`;
        } else if (q < 0) {
            factor += ` - ${Math.abs(q)}`;
        }
        
        return factor;
    }
    
    // Show the solution steps
    function showSolution(a, b, c, p1, q1, p2, q2) {
        solutionSteps.style.display = 'block';
        stepsList.innerHTML = '';
        
        // Step 1: Original equation
        const step1 = document.createElement('li');
        step1.innerHTML = `Start with the equation: <strong>${formatEquation(a, b, c)}</strong>`;
        stepsList.appendChild(step1);
        
        // Step 2: Factor pairs
        const step2 = document.createElement('li');
        step2.innerHTML = `Look for two numbers that multiply to give <strong>${a * c}</strong> and add up to <strong>${b}</strong>`;
        stepsList.appendChild(step2);
        
        // Step 3: Factorization
        const factor1 = formatFactor(p1, q1);
        const factor2 = formatFactor(p2, q2);
        
        const step3 = document.createElement('li');
        step3.innerHTML = `Factorize into: <strong>(${factor1})(${factor2}) = 0</strong>`;
        stepsList.appendChild(step3);
        
        // Step 4: Verification by FOIL
        const step4 = document.createElement('li');
        step4.innerHTML = `
            Verify using FOIL method:<br>
            <strong>(${factor1})(${factor2})</strong><br>
            = ${p1 * p2}x² + ${p1 * q2}x + ${p2 * q1}x + ${q1 * q2}<br>
            = ${p1 * p2}x² + ${p1 * q2 + p2 * q1}x + ${q1 * q2}<br>
            = ${formatEquation(a, b, c)}
        `;
        stepsList.appendChild(step4);
        
        // Find roots
        const root1 = q1 !== 0 ? -q1 / p1 : 0;
        const root2 = q2 !== 0 ? -q2 / p2 : 0;
        
        rootsDisplay.innerHTML = `
            From <strong>(${factor1}) = 0</strong>, we get x = ${root1}<br>
            From <strong>(${factor2}) = 0</strong>, we get x = ${root2}
        `;
        
        // Verify roots
        const verification1 = a * Math.pow(root1, 2) + b * root1 + c;
        const verification2 = a * Math.pow(root2, 2) + b * root2 + c;
        
        verificationDisplay.innerHTML = `
            For x = ${root1}:<br>
            ${a}(${root1})² + ${b}(${root1}) + ${c} = ${verification1.toFixed(2)}<br><br>
            For x = ${root2}:<br>
            ${a}(${root2})² + ${b}(${root2}) + ${c} = ${verification2.toFixed(2)}
        `;
    }
    
    // Event listeners
    updateBtn.addEventListener('click', updateEquation);
    checkBtn.addEventListener('click', checkFactorization);
    
    // Initialize
    updateEquation();
})();
</script>"""

# Display the HTML content
HTML(html_content)

NameError: name 'HTML' is not defined


## Graphical Solutions to Quadratic Equations
### Theory
#### Understanding the Parabola

When we solve quadratic equations in the form $ax^2 + bx + c = 0$, we're searching for values of $x$ that make the expression equal to zero. But what happens if we evaluate this expression for many different values of $x$?

If we plot these inputs and outputs on a coordinate plane, with $x$ on the horizontal axis and the result on the vertical axis, we get a distinctive U-shaped curve called a **parabola**. This graphical representation lets us visualize quadratic equations in a powerful way:

$$y = ax^2 + bx + c$$

Where $y$ represents the output value for any given input $x$.

#### Key Insights from Graphing Quadratics

The graph of a quadratic equation reveals several important features:

1. **X-intercepts**: The points where the parabola crosses the x-axis are precisely the solutions to the quadratic equation. At these points, $y = 0$, which means they satisfy our original equation $ax^2 + bx + c = 0$.

2. **Vertex** (turning point): The lowest point of the parabola (if $a > 0$) or highest point (if $a < 0$) indicates where the function changes direction. If this point lies exactly on the x-axis, it means the equation has a repeated root.

3. **Number of solutions**: A parabola can:
   - Cross the x-axis twice (two distinct real solutions)
   - Touch the x-axis once (one repeated real solution)
   - Never cross the x-axis (two complex solutions)

#### Completing the Square

Another useful way to express quadratic expressions is in "completed square form":

$$a(x + p)^2 + q$$

Where $a$, $p$, and $q$ are constants. This form provides immediate insight into the graph's shape and position:
- $a$ determines whether the parabola opens upward ($a > 0$) or downward ($a < 0$)
- $p$ tells us the horizontal shift of the vertex (it's at $x = -p$)
- $q$ gives us the y-coordinate of the vertex

Converting from standard form to completed square form is a technique called "completing the square," which is also the foundation for deriving the quadratic formula.

#### The Process of Completing the Square

To convert $ax^2 + bx + c$ to $a(x + p)^2 + q$:

1. Factor out the coefficient $a$ from the first two terms
2. Complete the square for the expression inside the parentheses
3. Add and subtract the necessary constant to maintain equality

### Examples
#### Example 1: When $a = 1$

Let's convert $x^2 + 3x - 4$ to completed square form:

**Step 1**: Since $a = 1$, we can proceed directly to completing the square.

**Step 2**: To complete the square for $x^2 + 3x$, we:
- Take half the coefficient of $x$: $\frac{3}{2}$
- Square it: $\left(\frac{3}{2}\right)^2 = \frac{9}{4}$
- Add and subtract this value to maintain equality:

$$x^2 + 3x - 4 = x^2 + 3x + \frac{9}{4} - \frac{9}{4} - 4$$

**Step 3**: Recognize the perfect square and simplify:

$$x^2 + 3x - 4 = \left(x + \frac{3}{2}\right)^2 - \frac{9}{4} - 4 = \left(x + \frac{3}{2}\right)^2 - \frac{25}{4}$$

So the completed square form is: $\left(x + \frac{3}{2}\right)^2 - \frac{25}{4}$

#### Example 2: When $a \neq 1$

Let's convert $5x^2 + 2x + 10$ to completed square form:

**Step 1**: Factor out the coefficient of $x^2$:

$$5x^2 + 2x + 10 = 5\left(x^2 + \frac{2}{5}x\right) + 10$$

**Step 2**: Complete the square inside the parentheses:
- Take half the coefficient of $x$: $\frac{1}{5}$
- Square it: $\left(\frac{1}{5}\right)^2 = \frac{1}{25}$
- Add and subtract the necessary value inside the parentheses:

$$5\left(x^2 + \frac{2}{5}x + \frac{1}{25} - \frac{1}{25}\right) + 10$$

**Step 3**: Rearrange and simplify:

$$5\left(x^2 + \frac{2}{5}x + \frac{1}{25}\right) - 5 \cdot \frac{1}{25} + 10$$
$$5\left(x + \frac{1}{5}\right)^2 - \frac{5}{25} + 10$$
$$5\left(x + \frac{1}{5}\right)^2 - \frac{1}{5} + 10$$
$$5\left(x + \frac{1}{5}\right)^2 + \frac{49}{5}$$

So the completed square form is: $5\left(x + \frac{1}{5}\right)^2 + \frac{49}{5}$

#### Applications of Completed Square Form

This form is particularly useful for:
1. Finding the vertex of the parabola directly
2. Determining the minimum or maximum value of a quadratic function
3. Understanding how transformations affect the graph
4. Deriving the quadratic formula
5. Solving certain types of equations where this form is more convenient

In the Leaving Cert exam, you may need to use completing the square specifically, so practicing this technique is essential.


In [ ]:
html_content = """
<div id="completed-square-explorer" style="font-family: 'Arial', sans-serif; max-width: 900px; margin: 0 auto; padding: 20px;">
    <h2 style="text-align: center; color: #2c3e50;">Completed Square Form Visualizer</h2>
    <div style="display: flex; flex-wrap: wrap; gap: 20px;">
        <div style="flex: 1; min-width: 300px;">
            <div style="margin-bottom: 20px;">
                <h3>Parameters in Completed Square Form:</h3>
                <p><strong>a(x+p)² + q</strong></p>
                
                <div style="margin-bottom: 15px;">
                    <label for="cs-a-value">a: <span id="cs-a-value-display">1</span></label>
                    <input type="range" id="cs-a-value" min="-3" max="3" value="1" step="0.1" style="width: 100%;">
                </div>
                
                <div style="margin-bottom: 15px;">
                    <label for="cs-p-value">p: <span id="cs-p-value-display">0</span></label>
                    <input type="range" id="cs-p-value" min="-5" max="5" value="0" step="0.1" style="width: 100%;">
                </div>
                
                <div style="margin-bottom: 15px;">
                    <label for="cs-q-value">q: <span id="cs-q-value-display">0</span></label>
                    <input type="range" id="cs-q-value" min="-10" max="10" value="0" step="0.1" style="width: 100%;">
                </div>
            </div>
            
            <div style="margin-top: 20px; padding: 15px; background-color: #f8f9fa; border-radius: 5px; box-shadow: 0 1px 3px rgba(0,0,0,0.1);">
                <h3 style="margin-top: 0; color: #2c3e50;">Equation Forms:</h3>
                <p><strong>Completed Square Form:</strong> <span id="cs-equation-display">a(x+p)² + q</span></p>
                <p><strong>Standard Form:</strong> <span id="standard-equation-display">ax² + bx + c = 0</span></p>
            </div>
            
            <div style="margin-top: 20px; padding: 15px; background-color: #f8f9fa; border-radius: 5px; box-shadow: 0 1px 3px rgba(0,0,0,0.1);">
                <h3 style="margin-top: 0; color: #2c3e50;">Key Features:</h3>
                <p><strong>Vertex:</strong> <span id="vertex-display">(0, 0)</span></p>
                <p><strong>X-intercepts:</strong> <span id="x-intercepts-display">None</span></p>
                <p><strong>Y-intercept:</strong> <span id="y-intercept-display">0</span></p>
                <p><strong>Line of Symmetry:</strong> x = <span id="symmetry-display">0</span></p>
                <p><strong>Discriminant:</strong> <span id="discriminant-display">0</span></p>
            </div>
        </div>
        
        <div style="flex: 2; min-width: 400px;">
            <div style="width: 100%; height: 400px; border: 1px solid #ddd; border-radius: 5px; box-shadow: 0 1px 3px rgba(0,0,0,0.1);">
                <canvas id="cs-graph-canvas" style="width: 100%; height: 100%;"></canvas>
            </div>
            
            <div style="margin-top: 20px; padding: 15px; background-color: #e8f4fc; border-radius: 5px; box-shadow: 0 1px 3px rgba(0,0,0,0.1);">
                <h3 style="margin-top: 0; color: #2c3e50;">Understanding the Completed Square Form:</h3>
                <p id="explanation-text">
                    The completed square form a(x+p)² + q reveals essential information about the parabola:
                    <ul>
                        <li><strong>a</strong> determines if the parabola opens upward (a > 0) or downward (a < 0), and affects how "steep" the curve is</li>
                        <li><strong>p</strong> tells us the horizontal position of the vertex: it's located at x = -p</li>
                        <li><strong>q</strong> gives us the vertical position of the vertex: it's the y-coordinate</li>
                    </ul>
                    Adjust the sliders to see how each parameter affects the graph!
                </p>
            </div>
        </div>
    </div>
</div>

<script>
(function() {
    // Get DOM elements
    const canvas = document.getElementById('cs-graph-canvas');
    const ctx = canvas.getContext('2d');
    
    const aSlider = document.getElementById('cs-a-value');
    const pSlider = document.getElementById('cs-p-value');
    const qSlider = document.getElementById('cs-q-value');
    
    const aValueDisplay = document.getElementById('cs-a-value-display');
    const pValueDisplay = document.getElementById('cs-p-value-display');
    const qValueDisplay = document.getElementById('cs-q-value-display');
    
    const csEquationDisplay = document.getElementById('cs-equation-display');
    const standardEquationDisplay = document.getElementById('standard-equation-display');
    
    const vertexDisplay = document.getElementById('vertex-display');
    const xInterceptsDisplay = document.getElementById('x-intercepts-display');
    const yInterceptDisplay = document.getElementById('y-intercept-display');
    const symmetryDisplay = document.getElementById('symmetry-display');
    const discriminantDisplay = document.getElementById('discriminant-display');
    
    // Set canvas dimensions to match its display size
    function resizeCanvas() {
        const displayWidth = canvas.clientWidth;
        const displayHeight = canvas.clientHeight;
        
        if (canvas.width !== displayWidth || canvas.height !== displayHeight) {
            canvas.width = displayWidth;
            canvas.height = displayHeight;
        }
    }
    resizeCanvas();
    window.addEventListener('resize', resizeCanvas);
    
    // Graph properties
    const graphProps = {
        xMin: -10,
        xMax: 10,
        yMin: -10,
        yMax: 10,
        gridSpacing: 1,
        padding: 30
    };
    
    // Convert from math coordinates to canvas coordinates
    function toCanvasX(x) {
        return ((x - graphProps.xMin) / (graphProps.xMax - graphProps.xMin)) * (canvas.width - 2 * graphProps.padding) + graphProps.padding;
    }
    
    function toCanvasY(y) {
        return canvas.height - (((y - graphProps.yMin) / (graphProps.yMax - graphProps.yMin)) * (canvas.height - 2 * graphProps.padding) + graphProps.padding);
    }
    
    // Convert from completed square form to standard form
    function completedSquareToStandard(a, p, q) {
        // a(x+p)² + q = a(x² + 2px + p²) + q
        //              = ax² + 2apx + ap² + q
        const stdA = a;
        const stdB = 2 * a * p;
        const stdC = a * (p * p) + q;
        
        return { a: stdA, b: stdB, c: stdC };
    }
    
    // Quadratic function in completed square form
    function completedSquare(x, a, p, q) {
        return a * Math.pow(x + p, 2) + q;
    }
    
    // Calculate features of the parabola
    function calculateFeatures(a, p, q) {
        // Convert to standard form
        const { b, c } = completedSquareToStandard(a, p, q);
        
        // Vertex
        const vertexX = -p;
        const vertexY = q;
        
        // X-intercepts (where y = 0)
        // Solve: a(x+p)² + q = 0
        //       (x+p)² = -q/a
        const discriminant = b * b - 4 * a * c;
        
        let xIntercepts = [];
        if (discriminant > 0) {
            const x1 = (-b + Math.sqrt(discriminant)) / (2 * a);
            const x2 = (-b - Math.sqrt(discriminant)) / (2 * a);
            xIntercepts = [x1, x2];
        } else if (discriminant === 0) {
            const x = -b / (2 * a);
            xIntercepts = [x];
        }
        
        // Y-intercept (where x = 0)
        const yIntercept = completedSquare(0, a, p, q);
        
        // Line of symmetry
        const symmetryLine = vertexX;
        
        return {
            vertex: { x: vertexX, y: vertexY },
            xIntercepts,
            yIntercept,
            symmetryLine,
            discriminant
        };
    }
    
    // Draw the grid
    function drawGrid() {
        ctx.strokeStyle = '#e1e1e1';
        ctx.lineWidth = 1;
        
        // Vertical grid lines
        for (let x = Math.ceil(graphProps.xMin); x <= graphProps.xMax; x += graphProps.gridSpacing) {
            ctx.beginPath();
            ctx.moveTo(toCanvasX(x), 0);
            ctx.lineTo(toCanvasX(x), canvas.height);
            ctx.stroke();
        }
        
        // Horizontal grid lines
        for (let y = Math.ceil(graphProps.yMin); y <= graphProps.yMax; y += graphProps.gridSpacing) {
            ctx.beginPath();
            ctx.moveTo(0, toCanvasY(y));
            ctx.lineTo(canvas.width, toCanvasY(y));
            ctx.stroke();
        }
        
        // Axes
        ctx.strokeStyle = '#000';
        ctx.lineWidth = 2;
        
        // X-axis
        ctx.beginPath();
        ctx.moveTo(toCanvasX(graphProps.xMin), toCanvasY(0));
        ctx.lineTo(toCanvasX(graphProps.xMax), toCanvasY(0));
        ctx.stroke();
        
        // Y-axis
        ctx.beginPath();
        ctx.moveTo(toCanvasX(0), toCanvasY(graphProps.yMin));
        ctx.lineTo(toCanvasX(0), toCanvasY(graphProps.yMax));
        ctx.stroke();
        
        // Axis labels
        ctx.fillStyle = '#000';
        ctx.font = '12px Arial';
        ctx.textAlign = 'center';
        
        // X-axis labels
        for (let x = Math.ceil(graphProps.xMin); x <= graphProps.xMax; x += graphProps.gridSpacing) {
            if (x !== 0) {  // Skip zero to avoid overlapping with y-axis label
                ctx.fillText(x.toString(), toCanvasX(x), toCanvasY(0) + 15);
            }
        }
        
        // Y-axis labels
        ctx.textAlign = 'right';
        for (let y = Math.ceil(graphProps.yMin); y <= graphProps.yMax; y += graphProps.gridSpacing) {
            if (y !== 0) {  // Skip zero to avoid overlapping with x-axis label
                ctx.fillText(y.toString(), toCanvasX(0) - 5, toCanvasY(y) + 4);
            }
        }
        
        // Origin label
        ctx.fillText("0", toCanvasX(0) - 5, toCanvasY(0) + 15);
    }
    
    // Draw the parabola
    function drawParabola(a, p, q) {
        ctx.strokeStyle = '#3498db';
        ctx.lineWidth = 3;
        ctx.beginPath();
        
        const step = (graphProps.xMax - graphProps.xMin) / canvas.width;
        
        for (let x = graphProps.xMin; x <= graphProps.xMax; x += step) {
            const y = completedSquare(x, a, p, q);
            
            if (y >= graphProps.yMin && y <= graphProps.yMax) {
                if (x === graphProps.xMin) {
                    ctx.moveTo(toCanvasX(x), toCanvasY(y));
                } else {
                    ctx.lineTo(toCanvasX(x), toCanvasY(y));
                }
            }
        }
        
        ctx.stroke();
    }
    
    // Draw special points and features
    function drawFeatures(a, p, q) {
        const features = calculateFeatures(a, p, q);
        
        // Vertex
        ctx.fillStyle = '#e74c3c';
        ctx.beginPath();
        ctx.arc(toCanvasX(features.vertex.x), toCanvasY(features.vertex.y), 5, 0, 2 * Math.PI);
        ctx.fill();
        
        // Draw label for vertex
        ctx.font = '12px Arial';
        ctx.fillText("Vertex", toCanvasX(features.vertex.x) + 10, toCanvasY(features.vertex.y) - 10);
        
        // X-intercepts
        ctx.fillStyle = '#9b59b6';
        features.xIntercepts.forEach(x => {
            ctx.beginPath();
            ctx.arc(toCanvasX(x), toCanvasY(0), 5, 0, 2 * Math.PI);
            ctx.fill();
        });
        
        // Y-intercept
        ctx.fillStyle = '#2ecc71';
        ctx.beginPath();
        ctx.arc(toCanvasX(0), toCanvasY(features.yIntercept), 5, 0, 2 * Math.PI);
        ctx.fill();
        
        // Line of symmetry
        ctx.setLineDash([5, 5]);
        ctx.strokeStyle = '#f39c12';
        ctx.lineWidth = 2;
        ctx.beginPath();
        ctx.moveTo(toCanvasX(features.symmetryLine), toCanvasY(graphProps.yMin));
        ctx.lineTo(toCanvasX(features.symmetryLine), toCanvasY(graphProps.yMax));
        ctx.stroke();
        ctx.setLineDash([]);
    }
    
    // Format equation strings
    function formatCompletedSquare(a, p, q) {
        let equation = '';
        
        if (a === 0) {
            return q.toString();
        }
        
        if (a === 1) {
            equation = '';
        } else if (a === -1) {
            equation = '-';
        } else {
            equation = a.toString();
        }
        
        if (p === 0) {
            equation += 'x²';
        } else {
            equation += `(x${p > 0 ? ' + ' + p : ' - ' + Math.abs(p)})²`;
        }
        
        if (q !== 0) {
            equation += q > 0 ? ` + ${q}` : ` - ${Math.abs(q)}`;
        }
        
        return equation;
    }
    
    function formatStandardForm(a, b, c) {
        let equation = '';
        
        if (a === 0) {
            if (b === 0) {
                return c.toString() + ' = 0';
            }
            
            if (b === 1) {
                equation = 'x';
            } else if (b === -1) {
                equation = '-x';
            } else {
                equation = `${b}x`;
            }
            
            if (c !== 0) {
                equation += c > 0 ? ` + ${c}` : ` - ${Math.abs(c)}`;
            }
            
            equation += ' = 0';
            return equation;
        }
        
        if (a === 1) {
            equation = 'x²';
        } else if (a === -1) {
            equation = '-x²';
        } else {
            equation = `${a}x²`;
        }
        
        if (b !== 0) {
            if (b === 1) {
                equation += ' + x';
            } else if (b === -1) {
                equation += ' - x';
            } else {
                equation += b > 0 ? ` + ${b}x` : ` - ${Math.abs(b)}x`;
            }
        }
        
        if (c !== 0) {
            equation += c > 0 ? ` + ${c}` : ` - ${Math.abs(c)}`;
        }
        
        equation += ' = 0';
        return equation;
    }
    
    // Format x-intercepts for display
    function formatXIntercepts(intercepts) {
        if (intercepts.length === 0) {
            return "None (no x-intercepts)";
        } else if (intercepts.length === 1) {
            return `x = ${intercepts[0].toFixed(2)} (one x-intercept)`;
        } else {
            return `x = ${intercepts[0].toFixed(2)} and x = ${intercepts[1].toFixed(2)}`;
        }
    }
    
    // Update the graph and information
    function updateGraph() {
        // Get current slider values
        const a = parseFloat(aSlider.value);
        const p = parseFloat(pSlider.value);
        const q = parseFloat(qSlider.value);
        
        // Calculate standard form coefficients
        const stdForm = completedSquareToStandard(a, p, q);
        
        // Calculate features
        const features = calculateFeatures(a, p, q);
        
        // Clear canvas
        ctx.clearRect(0, 0, canvas.width, canvas.height);
        
        // Draw grid and function
        drawGrid();
        drawParabola(a, p, q);
        drawFeatures(a, p, q);
        
        // Update displays
        aValueDisplay.textContent = a.toFixed(1);
        pValueDisplay.textContent = p.toFixed(1);
        qValueDisplay.textContent = q.toFixed(1);
        
        csEquationDisplay.textContent = formatCompletedSquare(a, p, q);
        standardEquationDisplay.textContent = formatStandardForm(stdForm.a, stdForm.b, stdForm.c);
        
        vertexDisplay.textContent = `(${features.vertex.x.toFixed(2)}, ${features.vertex.y.toFixed(2)})`;
        xInterceptsDisplay.textContent = formatXIntercepts(features.xIntercepts);
        yInterceptDisplay.textContent = features.yIntercept.toFixed(2);
        symmetryDisplay.textContent = features.symmetryLine.toFixed(2);
        discriminantDisplay.textContent = features.discriminant.toFixed(2);
    }
    
    // Event listeners for sliders
    aSlider.addEventListener('input', updateGraph);
    pSlider.addEventListener('input', updateGraph);
    qSlider.addEventListener('input', updateGraph);
    
    // Initial render
    updateGraph();
})();
</script>
"""

HTML(html_content)


## Deriving the Quadratic Formula

I mentioned previously that the completed square form can be used to derive the "minus b" formula that you've almost certainly already met at Junior Cert level. This derivation is on the syllabus but has never before been asked. We will go through it quickly before we discuss using it.

The quadratic formula we're aiming to derive is:

\begin{gather}
x = \frac{-b \pm \sqrt{b^2 - 4ac}}{2a}
\end{gather}

And we are starting from standard form:

\begin{gather}
ax^2 + bx + c = 0
\end{gather}

The first thing we want to do is to divide by $a$ to neaten things up slightly:

\begin{gather}
x^2 + \frac{bx}{a} + \frac{c}{a} = 0
\end{gather}

We now want to put it into completed square form, so we choose $d$ of the completed square form to be $\frac{b}{2a}$, but we get an extra term at the end:

\begin{gather}
\left(x+\frac{b}{2a}\right)^2 = x^2 + \frac{b}{2a}x + \frac{b}{2a}x + \frac{b^2}{4a^2}\\
\end{gather}

Therefore we put in a $-\frac{b^2}{4a^2}$ to deal with it:

\begin{gather}
\left(x+\frac{b}{2a}\right)^2 -\frac{b^2}{4a^2} = x^2 + \frac{b}{a}x + \frac{b^2}{4a^2} - \frac{b^2}{4a^2} = x^2 + \frac{b}{a}x\\
\end{gather}

Comparing this with our target equation, we're almost there, we just need to add a $c/a$ term:

\begin{gather}
\left(x+\frac{b}{2a}\right)^2 - \frac{b^2}{4a^2} + \frac{c}{a} = x^2 + \frac{b}{a} + \frac{c}{a} = 0
\end{gather}

And now we do some algebra to get the $x$ on its own:

\begin{align*}
\left(x+\frac{b}{2a}\right)^2 &= \frac{b^2}{4a^2} - \frac{c}{a} \\
x+\frac{b}{2a} &= \pm\sqrt{\frac{b^2}{4a^2} - \frac{c}{a}}
\end{align*}

And then continue to move things around to get things in a nicer form:

\begin{align*}
2ax + b &= \pm 2a \sqrt{\frac{b^2}{4a^2} - \frac{c}{a}} \\
&= \pm \sqrt{4a^2} \sqrt{\frac{b^2}{4a^2} - \frac{c}{a}} \\
&= \pm \sqrt{\frac{4a^2b^2}{4a^2} - \frac{4a^2c}{a}} \\
&= \pm \sqrt{b^2 - 4ac}
\end{align*}

This should be beginning to look familiar now. We move the last few pieces over and end up with our quadratic formula:

\begin{align*}
2ax + b &= \pm \sqrt{b^2 - 4ac} \\
2ax &= -b \pm \sqrt{b^2 - 4ac} \\
x &= \frac{-b \pm \sqrt{b^2 - 4ac}}{2a}
\end{align*}

The formula we just derived is **very** useful, because it allows us to jump directly to solutions for $x$ for a quadratic in $ax^2 + bx +c$ form without fiddling about with algebra. On occasion it's faster to factorise by inspection rather than go through the formula, especially if the factorisation is very obvious, but it's important to remember the quadratic formula every time there are roots to find.


## The Discriminant: Understanding the Nature of Quadratic Solutions

The discriminant is a powerful mathematical tool that reveals the nature of a quadratic equation's solutions before you even solve it. It acts as a "solution predictor" by examining what happens under the square root in the quadratic formula.

### Theory
#### What is the Discriminant?

For any quadratic equation in standard form $ax^2 + bx + c = 0$, the discriminant (denoted by $\Delta$) is defined as:

$$\Delta = b^2 - 4ac$$

This expression appears under the square root in the quadratic formula:

$$x = \frac{-b \pm \sqrt{b^2 - 4ac}}{2a} = \frac{-b \pm \sqrt{\Delta}}{2a}$$

#### The Three Scenarios

The value of the discriminant determines both the number and type of solutions:

##### Case 1: Positive Discriminant ($\Delta > 0$)
- **Algebraically**: The equation has two distinct real solutions
- **Graphically**: The parabola crosses the x-axis at two different points
- **Example**: For $x^2 - 5x + 6 = 0$:
  - $\Delta = (-5)^2 - 4(1)(6) = 25 - 24 = 1 > 0$
  - Solutions are $x = 2$ and $x = 3$ (two distinct real roots)
  - The parabola crosses the x-axis at $x = 2$ and $x = 3$

##### Case 2: Zero Discriminant ($\Delta = 0$)
- **Algebraically**: The equation has exactly one repeated real solution
- **Graphically**: The parabola touches the x-axis exactly once at its vertex
- **Example**: For $x^2 - 6x + 9 = 0$:
  - $\Delta = (-6)^2 - 4(1)(9) = 36 - 36 = 0$
  - The only solution is $x = 3$ (repeated twice)
  - The parabola's vertex is at $(3,0)$, just touching the x-axis

##### Case 3: Negative Discriminant ($\Delta < 0$)
- **Algebraically**: The equation has two complex solutions (not real numbers)
- **Graphically**: The parabola never intersects the x-axis
- **Example**: For $x^2 + 1 = 0$:
  - $\Delta = 0^2 - 4(1)(1) = -4 < 0$
  - Solutions are $x = \pm i$ (complex numbers)
  - The parabola sits entirely above the x-axis with vertex at $(0,1)$

#### Why Does This Matter?

Understanding the discriminant helps you:

1. **Predict solutions** without solving the entire equation
2. **Choose solution methods** appropriately
3. **Verify answers** by checking if they match the expected number of solutions
4. **Understand the graphical meaning** of quadratic equations

#### The Discriminant and Factorization

The discriminant also tells us about factorization possibilities:

- When $\Delta > 0$: The quadratic can be factored into two different linear factors with real coefficients
- When $\Delta = 0$: The quadratic can be factored as a perfect square: $(x - r)^2$
- When $\Delta < 0$: The quadratic cannot be factored with real coefficients

#### Complex Numbers and Beyond

When $\Delta < 0$, we enter the realm of complex numbers. Though these solutions don't appear on the standard coordinate plane, they're still valid mathematically. For instance, $x^2 + 1 = 0$ has solutions $x = \pm i$, where $i = \sqrt{-1}$ is the imaginary unit.

Complex solutions always appear in conjugate pairs (if $a + bi$ is a solution, then $a - bi$ is also a solution), reflecting the symmetry we see in real solutions as well.

#### Summary

The discriminant provides immediate insight into the nature of a quadratic equation's solutions:
- $\Delta > 0$: Two distinct real solutions
- $\Delta = 0$: One repeated real solution
- $\Delta < 0$: Two complex solutions (no real solutions)

This single value connects algebra, geometry, and number theory in a remarkably elegant way.


## Higher Order Polynomials
### Theory
The same way that "quadratic" refers to a polynomial of degree 2, we can have polynomials of degree 3 ("cubic"), or 4 ("quartic"). When graphed, they no longer have a parabola shape, but instead have extra turns - the higher order the polynomial is, the more turns it will have. Similar rules to quadratic functions dictate how higher order polynomials look when graphed. We will briefly consider these, as they are handy for visualising functions.

1. If the degree of a function is odd, then the "arms" (the parts of the function before and after the twists and turns where the function is constantly increasing/decreasing) will go in opposite directions. If the degree is even, the arms will go in the same direction.
2. If the leading coefficient (the constant term multiplied by the highest power of $x$) is positive, then the arm on the right hand side will go up. Otherwise, it will go down. To figure out what the left hand side arm is doing, check if the degree is odd or even.
3. The $y$-intercept, as learned in linear equations, holds here too: the constant term at the end that doesn't have any $x$ multiplied by it is the $y$-intercept, and this is the point that the graph cuts the $y$-axis.
4. Substituting $-x$ for $x$ mirrors the graph about the $y$-axis. Multiplying the whole equation by $-1$ (so changing $y$ to $-y$) flips the graph about the $x$-axis.
5. Roots that appear only once are the line crossing the $x$-axis. Roots that appear twice are the line touching the $x$-axis as it turns. Roots that appear three times are the line passing through the $x$-axis but which briefly flatten as it does so (to test this: graph $y = x^3$, which has the root $x=0$ three times, to see this shape in action).
6. If you multiply a polynomial by a constant, it makes the amplitude larger by that amount i.e., it stretches the graph vertically by that amount.
7. Local minima/maxima* are the lowest/highest points on a graph in their immediate area. We want these to be "local" as the arms of polynomials can stretch on to infinity whilst constantly increasing/decreasing, so overall, the minimum/maximum would be at $\pm\infty$.

*The singular form is local minimum/maximum

#### Cubic Expressions

For cubic expressions in the form:

\begin{gather}
ax^3 + bx^2 + cx + d
\end{gather}

Note: what the $a$, $b$, and $c$ are multiplied by has changed from the quadratic form, with $a$ now multiplying a cubic term.

If we take the discriminant ($\Delta = b^2 - 4ac$), we can check certain things about its roots:

* When $\Delta > 0$:

  * Three real roots
  * The line crosses the $x$-axis 3 times
* When $\Delta = 0$:

  * Three real roots but two are the same
  * The line crosses the $x$-axis once then just barely touches it as it turns
* When $\Delta < 0$:

  * One real root and two complex roots
  * The line crosses the $x$-axis once but twists and turns without approaching it again

In [7]:
# Example of how to use the modularized math resource in a Jupyter Book

# Import the render function
import sys
sys.path.append("..")
import modular_questions


# Define content for a new math concept (Derivatives)
derivatives_content = {
    "title": "Derivatives: Rates of Change",
    "intro_content": """
        <p>The derivative of a function is a measure of how a function changes as its input changes. 
        Mathematically, the derivative of a function f(x) is denoted as f'(x) or df/dx and is defined as:</p>
        <p style="text-align: center;" class="math-expression">
            <strong>f'(x) = lim<sub>h→0</sub> (f(x+h) - f(x))/h</strong>
        </p>
        <p>The derivative has various interpretations:</p>
        <ul>
            <li>The slope of the tangent line to the function at point x</li>
            <li>The instantaneous rate of change of the function at point x</li>
            <li>The sensitivity of the function output to small changes in input</li>
        </ul>
    """,
    "questions": [
        {
            "category": "scientific",
            "title": "Physics Application:",
            "content": """
                <p>A particle is moving along a straight line with its position at time t given by:</p>
                <p style="text-align: center;" class="math-expression">s(t) = 3t³ - 12t² + 9t - 1</p>
                <p>Find the derivative of this position function and interpret what it represents physically.</p>
                <p>At what time(s) is the particle's velocity zero?</p>
            """,
            "answer": """
                <p>The derivative of the position function gives us the velocity function:</p>
                <p class="math-expression">v(t) = s'(t) = 9t² - 24t + 9</p>
                
                <p>This represents the instantaneous velocity of the particle at time t.</p>
                
                <p>To find when velocity is zero, we solve:</p>
                <p class="math-expression">9t² - 24t + 9 = 0</p>
                <p class="math-expression">9(t² - 8/3t + 1) = 0</p>
                <p class="math-expression">t² - 8/3t + 1 = 0</p>
                
                <p>Using the quadratic formula:</p>
                <p class="math-expression">t = (8/3 ± √((8/3)² - 4))/2</p>
                <p class="math-expression">t = (8/3 ± √(64/9 - 4))/2</p>
                <p class="math-expression">t = (8/3 ± √(64/9 - 36/9))/2</p>
                <p class="math-expression">t = (8/3 ± √(28/9))/2</p>
                <p class="math-expression">t ≈ 0.423 or t ≈ 2.577</p>
                
                <p>So the particle's velocity is zero at approximately t = 0.423 seconds and t = 2.577 seconds. These represent moments when the particle momentarily stops before changing direction.</p>
            """
        },
        {
            "category": "engineering",
            "title": "Engineering Application:",
            "content": """
                <p>The cost of producing x units of a product is given by:</p>
                <p style="text-align: center;" class="math-expression">C(x) = 2000 + 50x - 0.1x² + 0.001x³</p>
                <p>Find the marginal cost function (the derivative of the cost function).</p>
                <p>What is the marginal cost when 100 units are produced? Interpret your result.</p>
            """,
            "answer": """
                <p>The marginal cost function is the derivative of the cost function:</p>
                <p class="math-expression">C'(x) = 50 - 0.2x + 0.003x²</p>
                
                <p>When x = 100 units:</p>
                <p class="math-expression">C'(100) = 50 - 0.2(100) + 0.003(100)²</p>
                <p class="math-expression">C'(100) = 50 - 20 + 0.003(10000)</p>
                <p class="math-expression">C'(100) = 50 - 20 + 30</p>
                <p class="math-expression">C'(100) = 60</p>
                
                <p>This means that when production is at 100 units, the cost of producing one additional unit is approximately $60. This is the instantaneous rate of change of the cost at the production level of 100 units.</p>
                
                <p>Engineers can use this information to make decisions about production levels and resource allocation, balancing the marginal cost against the marginal revenue to optimize profit.</p>
            """
        }
    ]
}

# Render the module
modular_questions.render_math_resource(derivatives_content)

In [ ]:
# Example of how to use the modularized MCQ system in a Jupyter Book

# Import the modular_mcq module
import sys
sys.path.append("..")
import modular_mcq

# Example 1: Create a quiz directly in Python code
quadratic_quiz = {
    "title": "Quadratic Discriminant Quiz",
    "questions": [
        {
            "text": "What is the discriminant of a quadratic equation ax² + bx + c = 0?",
            "options": [
                "a² - 4bc",
                "b² - 4ac",
                "2a + b",
                "b² + 4ac"
            ],
            "correctIndex": 1,
            "explanation": "The discriminant of a quadratic equation ax² + bx + c = 0 is given by the formula b² - 4ac. This value helps determine the nature of the roots of the equation.",
            "difficulty": "Basic"
        },
        {
            "text": "If the discriminant of a quadratic equation is negative, what can we conclude about its roots?",
            "options": [
                "The equation has two equal real roots",
                "The equation has two distinct real roots",
                "The equation has no real roots (two complex conjugate roots)",
                "The equation has one real root"
            ],
            "correctIndex": 2,
            "explanation": "When the discriminant (b² - 4ac) is negative, the quadratic equation has no real roots. Instead, it has two complex conjugate roots of the form α ± βi, where i is the imaginary unit.",
            "difficulty": "Intermediate"
        }
    ]
}

# Render the quiz
modular_mcq.render_mcq_quiz(quadratic_quiz, "quadratic_quiz_1")

# Example 2: Use the built-in example
modular_mcq.discriminant_quiz_example()

# Example 3: Load from a JSON file
# First, let's create a JSON file
import json
import os

derivatives_quiz = {
    "title": "Derivatives Assessment",
    "questions": [
        {
            "text": "What is the derivative of f(x) = x²?",
            "options": [
                "f'(x) = x",
                "f'(x) = 2x",
                "f'(x) = 2",
                "f'(x) = 0"
            ],
            "correctIndex": 1,
            "explanation": "The derivative of x² is 2x. Using the power rule, the derivative of x^n is n·x^(n-1).",
            "difficulty": "Basic"
        },
        {
            "text": "What is the derivative of f(x) = sin(x)?",
            "options": [
                "f'(x) = cos(x)",
                "f'(x) = -sin(x)",
                "f'(x) = tan(x)",
                "f'(x) = -cos(x)"
            ],
            "correctIndex": 0,
            "explanation": "The derivative of sin(x) is cos(x).",
            "difficulty": "Basic"
        }
    ]
}

# Save to a JSON file
with open("derivatives_quiz.json", "w") as f:
    json.dump(derivatives_quiz, f, indent=2)

# Load and render the quiz from the JSON file
modular_mcq.load_quiz_from_json("derivatives_quiz.json", "derivatives_quiz")

# Example 4: Create multiple quizzes with different IDs
integral_quiz = {
    "title": "Integration Techniques",
    "questions": [
        {
            "text": "What is the integral of f(x) = 2x?",
            "options": [
                "F(x) = x²",
                "F(x) = x² + C",
                "F(x) = 2x² + C",
                "F(x) = x² + 2C"
            ],
            "correctIndex": 1,
            "explanation": "The integral of 2x is x². Don't forget to add the constant of integration C!",
            "difficulty": "Basic"
        }
    ]
}

# Render multiple quizzes on the same page
#modular_mcq.render_mcq_quiz(quadratic_quiz, "quadratic_quiz_2")
modular_mcq.render_mcq_quiz(integral_quiz, "integral_quiz")
